<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Tutorial de Pytorch 9:  clasificador de imágenes**

Vamos a desarrollar un clasificador de imágenes utilizando una red neuronal convolutiva (CNN) en Pytorch que clasifique imágenes de correspondientes a diferentes estancias de una casas, en es caso: baños, cocinas, dormitorios, salones y comedores.


Comenzamos importando las librerías necesarias y preparando las imágenes del dataset. Separaremos las imágenes en dos conjuntos: uno de entrenamiento (80%) y otro de validación (20%). Para ello, utilizaremos la librería `random_split` de Pytorch.


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch import nn
import torch.optim as optim

# Transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar las imágenes a 224x224
    transforms.ToTensor(),  # Convertir las imágenes a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizar
])

# Cargar el dataset completo
full_dataset = ImageFolder(root='data/house_rooms', transform=transform)

from torch.utils.data import random_split

# Tamaños para entrenamiento y validación
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Dividir el dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

Quizá te haya llamado la atención los valores que hemos puesto en la normalización de las imágenes. Estos valores son la media y la desviación estándar de las imágenes del dataset ImageNet, que es un dataset muy grande y muy utilizado en el mundo del deep learning. Estos valores son los que se suelen utilizar para normalizar imágenes en tareas de clasificación de imágenes.

In [3]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

Para crear nuestro modelo no vamos a instanciar explícitamente cada capa de la red neuronal, sino que vamos a utilizar la clase `nn.Sequential` de Pytorch. Esta clase nos permite crear una red neuronal secuencial, es decir, una red neuronal en la que las capas se van aplicando una detrás de otra. En nuestro caso, vamos a crear una red neuronal con dos capas convolutivas, dos capas de pooling y dos capas completamente conectadas.

In [17]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, padding=1),  # Capa convolutiva
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(16, 32, 3, padding=1),  # Otra capa convolutiva
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),  # Aplanamos los tensores para la capa completamente conectada
    nn.Linear(32 * 56 * 56, 512),  # Ajustamos el tamaño según el tamaño de la imagen después de las capas convolutivas y de pooling
    nn.ReLU(),
    nn.Linear(512, 5)  # 5 categorías
)

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Creamos el bucle de entrenamiento y validación de la red neuronal. En cada iteración del bucle de entrenamiento, vamos a calcular el error de la red neuronal y vamos a actualizar los pesos de la red neuronal utilizando el algoritmo de optimización `Adam`. En cada iteración del bucle de validación, vamos a calcular el error y la precisión de la red neuronal en el conjunto de validación.

In [19]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Ponemos el modelo en modo de entrenamiento
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Época {epoch+1}, Pérdida de Entrenamiento: {round(running_loss/len(train_loader),3)}")
    
    # Evaluación con el conjunto de validación
    model.eval()  # Ponemos el modelo en modo de evaluación
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Desactivamos el cálculo de gradientes para la evaluación
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Época {epoch+1}, Pérdida de Validación: {round(val_loss/len(val_loader),3)}")
    print(f"Época {epoch+1}, Precisión de Validación: {round(100 * correct / total,3)}%")
    print("--------------------------------------------------")


Época 1, Pérdida de Entrenamiento: 2.22
Época 1, Pérdida de Validación: 1.424
Época 1, Precisión de Validación: 37.714%
--------------------------------------------------
Época 2, Pérdida de Entrenamiento: 1.297
Época 2, Pérdida de Validación: 1.328
Época 2, Precisión de Validación: 44.381%
--------------------------------------------------
Época 3, Pérdida de Entrenamiento: 0.778
Época 3, Pérdida de Validación: 1.379
Época 3, Precisión de Validación: 47.333%
--------------------------------------------------
Época 4, Pérdida de Entrenamiento: 0.188
Época 4, Pérdida de Validación: 1.895
Época 4, Precisión de Validación: 42.667%
--------------------------------------------------
Época 5, Pérdida de Entrenamiento: 0.062
Época 5, Pérdida de Validación: 1.879
Época 5, Precisión de Validación: 44.952%
--------------------------------------------------
